# Repackager

Create a package from a list of good files instead of a directory

In [5]:
from nbmod import nbloader
from Packager import package_df
from Clusterer import add_cluster_column
import pandas as pd
import os

Using TensorFlow backend.


In [6]:
CSV_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSgGCk9pdnXBQoIWNJTLwRcjScrFqPJTWujekS9JXrUsqc3VKpIkkFyVkJrc4B2riy68_YzOIT-yQcY/pub?gid=0&single=true&output=csv"
IMAGE_DIR = "./sample-images/cropped"

In [7]:
def get_df():
    return pd.read_csv(CSV_URL)

#get_df().head()

In [8]:
def get_good_df(initials):
    df = get_df()
    return df[df[f"{initials.lower().strip()}_good"] == True]
    
# get_good_df("jk").head()

In [9]:
class Package:
    
    def __init__(self, pkg_dir):
        self.pkg_dir = pkg_dir
        
    def clusters(self):
        return pd.read_csv(os.path.join(self.pkg_dir, "clusters.csv"))
    
    def cluster(self, cluster_id):
        ret = self.clusters()
        return ret[ret["cluster"].apply(lambda x: str(x)) == str(cluster_id)]
    


In [10]:
# Package("./out/packages-firstpass/2012m1_ref134_ex1/").clusters()

In [11]:
# Package("./out/packages-firstpass/2012m1_ref134_ex1/").cluster(4)

In [12]:
def get_mega_cluster(initials):
    good_df = get_good_df(initials)
    good_df["cluster"] = "mega"
    return good_df
#get_mega_cluster("cd")

In [13]:
def package_good(initials, dst_dir):
    
    """
        Re-cluster a user's selected 'good' images, and then re-cluster
        everything as one big set
    """
    
    dst_dir = os.path.join(dst_dir, initials)
    try:
        os.makedirs(dst_dir)
    except Exception as e:
        #print(f"Error: {e}")
        pass
    
    good_df = get_good_df(initials)
    mega_df = pd.DataFrame()
    row_sum = 0
    for i, row in good_df.iterrows():
        row_df = Package(f"./out/packages-firstpass/{row['folder_name']}/").cluster(row["cluster_id"])
        row_sum += len(row_df)

        print(f"Reclustering {row_sum} images in folder:{row['folder_name']} cluster:{row['cluster_id']}")
        row_df = add_cluster_column(row_df)
        row_dst = os.path.join(
                os.path.join(dst_dir, row['folder_name']),
                str(row["cluster_id"])
            )
        package_df(row_df, row_dst)
        
        row_df.to_csv(os.path.join(row_dst, "clusters.csv"), index=False)
        
        if mega_df.empty:
            mega_df = row_df.copy()
        else:
            mega_df = pd.concat([mega_df, row_df])
    
    print (row_sum)
    print (len(mega_df))
    mega_df["cluster"] = f"all" 
    
    ret = pd.concat([mega_df, mega_df2])
    
    package_df(ret, os.path.join("mega", dst_dir))
    
    return ret


In [15]:
#NBMODULE_IGNORE

# package_good("cd", "./out/packages-grading/1")
